In [1]:
from urllib.parse import urlencode

import numpy as np
import pandas as pd

In [2]:
def request_data(params):
    url = "https://dev.oec.world/olap-proxy/data.csv?{}".format(urlencode(params))
    return pd.read_csv(url)

In [3]:
# Sum of Exports by Country and HS92 Level 4 Code between 2016 and 2018
df_trade= pd.read_csv("https://app-tiger.oec.world/olap-proxy/data.csv?cube=trade_i_baci_a_92&measures=Trade%20Value&drilldowns=Exporter%20Country,HS4&Year=2016,2017,2018")

# World Population by Country for Year 2018
df_wdi = pd.read_csv("https://app-tiger.oec.world/olap-proxy/data.csv?cube=indicators_i_wdi_a&measures=Measure&drilldowns=Country&Indicator=SP.POP.TOTL&Year=2018")

In [4]:
#df_trade.info()
print(df_trade)
df_wdi.info()

       Country ID    Country  HS4 ID                HS4  Trade Value
0           afago     Angola   10101             Horses        615.0
1           afago     Angola   10102             Bovine      23491.0
2           afago     Angola   10103               Pigs        593.0
3           afago     Angola   10104    Sheep and Goats       3264.0
4           afago     Angola   10105            Poultry       9879.0
...           ...        ...     ...                ...          ...
201595      saven  Venezuela  219702             Prints     619426.0
201596      saven  Venezuela  219703         Sculptures    9290086.0
201597      saven  Venezuela  219704     Revenue Stamps       1434.0
201598      saven  Venezuela  219705  Collector's Items    1414813.0
201599      saven  Venezuela  219706           Antiques      12908.0

[201600 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
-

In [5]:
df = df_trade.copy()

# Countries with more than 1M habitants
df_population = df_wdi[df_wdi['Measure'] > 1000000]
# Products with more than $1.5B in global exports between 2016-2018
df_products = df.groupby('HS4 ID')['Trade Value'].sum().reset_index()
df_products = df_products[df_products['Trade Value'] > 3*500000000]
# Countries with more than $3B in global exports between 2016-2018
df_countries = df.groupby('Country ID')['Trade Value'].sum().reset_index()
df_countries = df_countries[df_countries['Trade Value'] > 3*1000000000]

df_filter  = df[
  (df['Country ID'].isin(df_population['Country ID'])) &
  (df['Country ID'].isin(df_countries['Country ID'])) & 
  (df['HS4 ID'].isin(df_products['HS4 ID']))
]



In [6]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136787 entries, 0 to 201599
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Country ID   136787 non-null  object 
 1   Country      136787 non-null  object 
 2   HS4 ID       136787 non-null  int64  
 3   HS4          136787 non-null  object 
 4   Trade Value  136787 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 6.3+ MB


In [7]:
df_pivot = pd.pivot_table(df_filter, index=['Country'],
                                     columns=['HS4 ID'],
                                     values='Trade Value')\
             .reset_index()\
             .set_index('Country')\
             .dropna(axis=1, how="all")\
             .fillna(0)\
             .astype(float)

print(df_pivot)

HS4 ID            10101      10102        10103      10104       10105   \
Country                                                                   
Afghanistan         98.0   725494.0          0.0  3317969.0         0.0   
Albania            965.0    18655.0          0.0    40350.0     87581.0   
Algeria         359367.0        0.0          0.0        0.0       263.0   
Angola             615.0    23491.0        593.0     3264.0      9879.0   
Argentina    103130398.0  2823338.0     428327.0   100035.0   3564490.0   
...                  ...        ...          ...        ...         ...   
Venezuela       473153.0    80672.0          0.0        0.0      1530.0   
Vietnam        1239500.0  9621365.0  135860396.0        0.0   2436708.0   
Yemen             4804.0        0.0          0.0   519121.0         0.0   
Zambia           52153.0   167489.0     102844.0        0.0  12152582.0   
Zimbabwe        207552.0    95098.0      11813.0     2011.0   2586678.0   

HS4 ID           10106  

In [12]:
"""Revealed Comparative Advantage (RCA) module

The Revealed Comparative Advantage is an index introduced by Balassa (1965),
which is used to evaluate the main products to be exported by a country,
and their comparative advantages in relation to the level of world exports
(Hidalgo et al., 2007).
"""
import numpy as np
import pandas as pd
def rca(tbl: pd.DataFrame) -> pd.DataFrame:
    """Calculates the Revealed Comparative Advantage (RCA) for a pivoted matrix.

    It is important to note that even though the functions do not use a
    parameter in relation to time, the data used for the calculations must
    be per period; for example working with World Exports for the year 2020.
    Also, the index always has to be a geographic level.

    Arguments:
        tbl (pandas.DataFrame) -- A pivoted table using a geographic index,
            columns with the categories to be evaluated and the measurement of
            the data as values.

    Returns:
        (pandas.DataFrame) -- RCA matrix with real values.
    """
    # fill missing values with zeros
    tbl = tbl.fillna(value=0)

    col_sums = tbl.sum(axis=1) # 1:columns
    col_sums = col_sums.to_numpy().reshape((len(col_sums), 1))

    rca_numerator = np.divide(tbl, col_sums)
    row_sums = tbl.sum(axis=0) # 0:index

    total_sum = tbl.sum().sum()
    rca_denominator = row_sums / total_sum
    rcas = rca_numerator / rca_denominator

    return rcas

In [10]:
from typing import Optional, Tuple

def complexity(rca: pd.DataFrame,
               iterations: int = 20,
               drop: Optional[bool] = True) -> Tuple[pd.Series, pd.Series]:
    """Calculates Economic Complexity Index (ECI) and Product Complexity
    Index (PCI) from a RCA matrix.

    Note that to display the resulting values, the series must be transformed
    into a dataframe in an tidy format as shown below:

        eci_value, pci_value = complexity(rca)
        eci = eci_value.to_frame(name="ECI").reset_index()
        pci = pci_value.to_frame(name="PCI").reset_index()

    Args:
        rcas (pd.DataFrame) -- Pivotted RCA matrix.
        iterations (int, optional) -- Limit of recursive calculations for
            kp and kc. Default value: 20.
        drop (bool, optional) -- Boolean to ensure that returns include NaN
            values. Default value: True.

    Returns:
        ((pd.Series, pd.Series)) -- A tuple of ECI and PCI values.
    """
    # Binarize rca input
    rcas = pd.DataFrame(rca.copy())
    rcas[rcas >= 1] = 1
    rcas[rcas < 1] = 0

    # drop columns / rows only if completely nan
    rcas_clone = rcas.copy()
    rcas_clone = rcas_clone.dropna(how="all")
    rcas_clone = rcas_clone.dropna(how="all", axis=1)

    if rcas_clone.shape != rcas.shape:
        logger.warning("RCAs contain columns or rows that are entirely comprised of NaN values.")
    if drop:
        rcas = rcas_clone
    
    print(rcas)
    
    kp = rcas.sum(axis=0) #sum columns
    kc = rcas.sum(axis=1) #sum rows

    print(kc)
    kp0 = kp.copy()
    kc0 = kc.copy()

    
    for i in range(1, iterations):
        kc_temp = kc.copy()
        kp_temp = kp.copy()
        kp = rcas.T.dot(kc_temp) / kp0
        if i < (iterations - 1):
            kc = rcas.dot(kp_temp) / kc0

    geo_complexity = (kc - kc.mean()) / kc.std()
    prod_complexity = (kp - kp.mean()) / kp.std()

    return geo_complexity, prod_complexity

In [13]:
rca = rca(df_pivot)
print(rca)
ECI, PCI = complexity(rca)

HS4 ID         10101     10102     10103     10104     10105      10106   \
Country                                                                    
Afghanistan  0.000164  0.369302  0.000000  9.026971  0.000000   9.139085   
Albania      0.000804  0.004732  0.000000  0.054699  0.063779   5.388141   
Algeria      0.020650  0.000000  0.000000  0.000000  0.000013   0.000156   
Angola       0.000034  0.000390  0.000021  0.000289  0.000471   0.078006   
Argentina    3.600635  0.030018  0.009875  0.005685  0.108810   0.238170   
...               ...       ...       ...       ...       ...        ...   
Venezuela    0.034015  0.001766  0.000000  0.000000  0.000096   0.098860   
Vietnam      0.011456  0.027080  0.829185  0.000000  0.019691   0.596998   
Yemen        0.006406  0.000000  0.000000  1.126684  0.000000   0.000000   
Zambia       0.010761  0.010524  0.014013  0.000000  2.192407   0.391491   
Zimbabwe     0.117857  0.016445  0.004430  0.001859  1.284257  12.021238   

HS4 ID     

In [14]:
ECI.sort_values(ascending=False)

Country
Japan               2.389178
Chinese Taipei      2.183193
Switzerland         2.085713
South Korea         2.019607
Germany             1.984514
                      ...   
Nigeria            -1.648391
Papua New Guinea   -1.738730
Iraq               -1.870009
South Sudan        -2.240257
Chad               -2.547961
Length: 148, dtype: float64